In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from numpy import mean

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect


In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base=automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurements=Base.classes.measurement
Station=Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session=Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
first_row = session.query(Measurements).first()
first_row.__dict__

In [ ]:
first_row = session.query(Station).first()
first_row.__dict__

In [ ]:
# Find the most recent date in the data set.
recent_date=session.query(Measurements.date)\
    .order_by(Measurements.date.desc()).first()
#convert query result into date
date_time = dt.datetime.strptime(recent_date[0], '%Y-%m-%d')

#get date ranges for quey 
date1=date_time.date()
date2=date1-dt.timedelta(days=365)

print(f"{date1} - {date2}")


In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
prcp=session.query(Measurements.date, Measurements.prcp)\
    .order_by(Measurements.date.desc())\
    .filter(Measurements.date<= date1)\
    .filter(Measurements.date >= date2)\
    .all()
prcp
# Starting from the most recent data point in the database. 
# Calculate the date one year from the last date in data set.
# Perform a query to retrieve the data and precipitation scores
# Save the query results as a Pandas DataFrame  
precipitation=pd.DataFrame(prcp)
precipitation

In [ ]:
#set the index to the date colum
precipitation.dropna(inplace=True)
precipitation.set_index('date', inplace=True)
# Sort the dataframe by date
precipitation.sort_values('date',ascending= True, inplace=True)

In [ ]:
precipitation

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
precipitation.reset_index(inplace=True)

In [ ]:
date_list=[]
date_list.append(date2.strftime('%Y-%m-%d'))
d=date2
for x in range(12):
    d= d+dt.timedelta(days=28)
    s=d.strftime('%Y-%m-%d')
    date_list.append(s)
    



In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
plt.figure(figsize=(10,6))
pyplot_bar=plt.bar(precipitation['date'], precipitation['prcp'], width= 3)
ax = plt.gca()
ax.set_xticks(date_list)
plt.xticks(rotation='vertical')
plt.title('Honolulu precipitation')
plt.xlabel('date')
plt.ylabel('Precipitation')
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precipitation.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
stations = session.query(Station.station).count()
stations

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
count_ = func.count('*')
most_active_station = session.query(Measurements.station, count_)\
        .group_by(Measurements.station)\
        .order_by(count_.desc())\
        .all()

most_active_station

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
station=session.query(Station.name)\
    .filter(Station.station == 'USC00519281').all()

min_temp=session.query(func.min(Measurements.tobs))\
    .filter(Measurements.station =='USC00519281')\
    .all()

max_temp=session.query(func.max(Measurements.tobs))\
    .filter(Measurements.station =='USC00519281')\
    .all()

avg_temp=session.query(func.avg(Measurements.tobs))\
    .filter(Measurements.station =='USC00519281')\
    .all()

print(f"station: {station[0][0]}\nMin temperature: {min_temp[0][0]} ºF\n\
Max temperature: {max_temp[0][0]} ºF\nAvg Temperature: {avg_temp[0][0]} ºF")


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temp=session.query(Measurements.date, Measurements.tobs)\
    .filter(Measurements.station =='USC00519281')\
    .filter(Measurements.date<=date1)\
    .filter(Measurements.date >=date2)\
    .all()
temp=pd.DataFrame(temp)
temp.head()

In [ ]:
x = temp['tobs']
plt.hist(x, bins = 12)
plt.xticks(rotation='vertical')
plt.title('WAIHEE Temperature Observation Data (TOBS)')
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.show()

# Close session

In [ ]:
# Close Session
session.close()